In [ ]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch_dsl import Search
from datetime import datetime
from tqdm import tqdm
import json

In [ ]:
today = datetime.now().strftime('%Y-%m-%d')

languages = ['in', 'en']

In [ ]:
for lang in languages:
    es = Elasticsearch(
        hosts = [{'host': 'localhost', 'port': 9200}],
        connection_class = RequestsHttpConnection,
    )
    s = Search(using=es, index='twitter_raw').filter("term", lang__keyword = lang)\
    .source(['data_text', 'retweet_text_full'])
    results = []
    for hit in tqdm(s.scan()):
        results.append(hit.to_dict())
        
    print(lang, len(results))
    
    with open(f'{today}-twitter-dump-{lang}.json', 'w') as fopen:
        json.dump(results, fopen)

In [1]:
from glob import glob

files = glob('*-twitter*.json')
files

['2020-02-22-twitter-dump-en.json', '2020-02-22-twitter-dump-in.json']

In [4]:
import boto3

bucketName = 'malaya-dataset'

for file in files:
    outPutname = f"dumping/twitter/{file}"

    s3 = boto3.client('s3')
    s3.upload_file(file,bucketName,outPutname)

2020-02-22-twitter-dump-en.json
2020-02-22-twitter-dump-in.json
